In [41]:
import pandas as pd
df = pd.read_csv("/home/ivana/data/all_data_info.csv")
seed = 123
df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [186]:
x = list(df['artist'].value_counts())
print(len([a for a in x if a >= 500]))
# len(set(x)) #---> ukupan broj umjetnika

12


In [163]:
# train, validation, test --- 80, 10, 10
num_train = 400
num_val = 50
num_test = num_val
num_samples = num_train + num_val + num_test
b_size = 60

#lista umjetnika koje ćemo promatrati
temp = df['artist'].value_counts()
threshold = num_samples
artists = temp[temp >= threshold].index.tolist()
print(artists)
num_artists = len(artists)
print(str(num_artist))

train_dfs = []
val_dfs = []
test_dfs = []

for a in artists:
    # PROVJERI KASNIJE ŠTA JE S NA=TRUE
    tmp = df[df['artist'].str.startswith(a)].sample(n=num_samples, random_state=seed)
    # print(tmp.shape)
    t_df = tmp.sample(n=num_train, random_state=seed)
    rest_df = tmp.loc[~tmp.index.isin(t_df.index)] # uzmi komplement od t_df
    # print(rest_df.shape)
    v_df = rest_df.sample(n=num_val, random_state=seed)
    te_df = rest_df.loc[~rest_df.index.isin(v_df.index)]
    
    train_dfs.append(t_df)
    val_dfs.append(v_df)
    test_dfs.append(te_df)


# print(type(train_df))
# print(type(train_dfs))
train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

['Pierre-Auguste Renoir', 'Marc Chagall', 'Zdislav Beksinski', 'John Singer Sargent', 'Giovanni Battista Piranesi', 'Ivan Aivazovsky', 'Ivan Shishkin', 'Rembrandt', 'Pablo Picasso', 'Gustave Dore', 'Albrecht Durer', 'Ilya Repin']
12


In [60]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential
from keras.datasets import cifar10
import numpy as np
import os

In [135]:
mean_resnet = np.array([0.485, 0.456, 0.406])
std_resnet = np.array([0.229, 0.224, 0.225])

def center_crop(img, center_crop_size):
    assert img.shape[2] == 3
    centerw, centerh = img.shape[0] // 2, img.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    return img[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

# https://jkjung-avt.github.io/keras-image-cropping/
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length, random_crop=True):
    '''
    Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator
    '''
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            if random_crop == True:
                batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
            else:
                batch_crops[i] = center_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)


In [160]:
datagen = ImageDataGenerator(
            horizontal_flip=True)

img = load_img('data/train_2/2.jpg')
img2 = load_img('data/train_2/20.jpg')
# img.show()
x = img_to_array(img)
# print(x.shape)
x = x.reshape((1,)+x.shape)
# print(x.shape)
x2 = img_to_array(img2)
x2 = x2.reshape((1,)+x2.shape)
print('x2', x2.shape)
X = [x, x2]

train_batch = datagen.flow(X, batch_size=1)
train_crops = crop_generator(train_batch, 224, False)

batch_x, batch_y = next(train_crops)
print(batch_x.shape)
print(batch_y.shape)
batch_x = batch_x.reshape(224,224,3)
print(batch_x.shape)
cropped_image = array_to_img(batch_x)
cropped_image.show()

x2 (1, 800, 591, 3)
(1, 224, 224, 3)
(1, 800, 591, 3)
(224, 224, 3)


In [20]:
datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

In [14]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [47]:
model = Sequential()
#nn.Conv2d(3, 32, kernel_size=3 +, stride=2 +, padding=1), # -> 112
model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=input_shape)) #kernel_size=3, stride=2, padding=1
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=input_shape)) #kernel_size=3, stride=2, padding=1
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
#nn.Linear(6272, 4*num_artists),
model.add(Dense(4*num_artists, input_shape=(6272,)))
model.add(Activation('relu'))
#nn.Linear(4*num_artists, num_artists)
model.add(Dense(num_artists, input_shape=(4*num_artists,)))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
# print(model_type)

Learning rate:  0.001
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 15, 15, 32)        896       
_________________________________________________________________
activation_33 (Activation)   (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 15, 15, 32)        128       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 3, 3, 32)          9248      
_________________________________________________________________
activation_34 (Activation)   (None, 3, 3, 32)          0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 3, 3, 32)         

In [48]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
callbacks = [checkpoint, lr_reducer, lr_scheduler] 

In [ ]:
datagen.fit(x_train)

In [33]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_test, y_test),
    epochs=epochs,
    verbose=1,
    workers=4,
    callbacks=callbacks)

ValueError: Error when checking target: expected activation_19 to have 4 dimensions, but got array with shape (10000, 10)

In [ ]:
# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])